In [ ]:
%pip install pandas scikit-learn seaborn plotly

In [ ]:
import pandas as pd

df = pd.read_csv("../datasets/analisis_presupuestario_salud.csv", delimiter=",")
df

In [ ]:
# import seaborn as sns

# sns.pairplot(df)

In [ ]:
# df.filter(like="").columns

In [ ]:
YEARS = range(2018, 2024)

EXCLUDED_VARIABLES = ["capitulo", "nombre_capitulo"]


def generate_scatter_matrix(df):
    analysis_variables = list(filter(lambda x: x not in EXCLUDED_VARIABLES, df.columns))

    for year in YEARS:
        plot = sns.pairplot(
            df[df["año"] == year][analysis_variables],
            kind="reg",
            diag_kind="kde",
            plot_kws={"line_kws": {"color": "red"}},
            diag_kws={"color": "red"},
        )

        for ax in plot.axes.flatten():
            ax.set_xlabel(ax.get_xlabel(), rotation=45)
            ax.set_ylabel(ax.get_ylabel(), rotation=45)
            ax.yaxis.get_label().set_horizontalalignment("right")

        plot.figure.suptitle(f"AÑO {year}", y=1.01, fontsize=16)
        plot.savefig(f"../plots/clustering/{year}.png", dpi=65)


# generate_scatter_matrix(df)

In [ ]:
import plotly.express as px

EXCLUDED_VARIABLES = ["capitulo", "nombre_capitulo"]


def generate_corr_matrices(df):
    analysis_variables = list(filter(lambda x: x not in EXCLUDED_VARIABLES, df.columns))

    for year in YEARS:
        corr_matrix = df[df["año"] == year][analysis_variables].corr()

        fig = px.imshow(
            corr_matrix,
            text_auto=True,
            color_continuous_scale="Plasma_r",
            aspect="auto",
            labels=dict(color="Correlación"),
            title=year,
        )
        fig.update_layout(height=1500, width=2500)
        fig.update_xaxes(tickangle=-45)
        fig.update_yaxes(tickangle=-45)

        fig.write_image(f"../plots/clustering/corr_matrix/{year}.png", scale=3)


# generate_corr_matrices(df)

In [ ]:
df.info()

In [ ]:
from sklearn.preprocessing import StandardScaler

scalable_columns = list(
    filter(lambda x: x not in ["año", "capitulo", "nombre_capitulo"], df.columns)
)

filled_df = df[scalable_columns].fillna(df[scalable_columns].mean())

scaler = StandardScaler()
scaled_features = scaler.fit_transform(filled_df[scalable_columns])
scaled_features

In [ ]:
scaled_df = pd.DataFrame(scaled_features)
scaled_df

for col, scaled_col in zip(df[scalable_columns].columns, scaled_df.columns):
    scaled_df.rename(columns={scaled_col: f"{col}_scaled"}, inplace=True)

scaled_df

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
import plotly.graph_objects as go

pca = PCA().fit(scaled_features)

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=np.arange(1, len(pca.explained_variance_ratio_.cumsum()) + 1),
        y=pca.explained_variance_ratio_.cumsum(),
        mode="lines+markers",
        name="Cumulative Explained Variance",
        line=dict(color="royalblue"),
    )
)

fig.update_layout(
    title="Varianza Explicada Acumulada (PCA)",
    xaxis_title="Componentes Principales",
    yaxis_title="Varianza Explicada Acumulada",
    xaxis=dict(range=[0, 47], tickmode="linear"),
    yaxis=dict(range=[0, 1.2]),
    template="plotly",
    height=800,
    width=1400,
)

fig.show()

In [ ]:
import plotly.express as px

N = 17
pca = PCA(n_components=N)

pca_result = pca.fit_transform(scaled_features)

fig = px.scatter(
    pca_result,
    x=pca_result[:, 0],
    y=pca_result[:, 1],
    title="PCA - Primeras dos componentes principales",
    labels={"x": "PCA 1", "y": "PCA 2"},
)
fig.update_layout(height=800, width=1400)
fig.show()

In [ ]:
from sklearn.cluster import KMeans

k_values = range(1, 11)

intertias = []

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42).fit(pca_result)
    intertias.append(kmeans.inertia_)

intertias

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=list(k_values),
        y=intertias,
        mode="lines+markers",
        marker=dict(size=8, color="blue"),
        name="Inercia",
    )
)

fig.update_layout(
    title="Método del codo para determinar k óptimo",
    xaxis_title="Cantidad de clústers (k)",
    yaxis_title="Inercia (suma de distancias cuadradas)",
    xaxis=dict(tickmode="linear"),
    template="plotly",
    height=800,
    width=1400,
)

fig.show()

In [ ]:
N_OPTIMAL = 3

kmeans_pca = KMeans(n_clusters=N_OPTIMAL, random_state=42)
kmeans_pca.fit(pca_result)

len(kmeans_pca.labels_)

In [ ]:
pca_df = pd.DataFrame(pca_result)
pca_df.columns = [f"pca_{i+1}" for i in range(len(pca_df.columns))]
pca_df["cluster"] = kmeans_pca.labels_.astype(str)

pca_df

In [ ]:
anio_servicio_cluster_df = pd.concat(
    [
        df[
            [
                "año",
                "capitulo",
                "nombre_capitulo",
                "gastos_en_personal",
                "total_gasto",
                "total_fonasa",
                "suicidio_ideacion",
                "suicidio_intento",
                "urgencias_lesiones_autoinflingidas",
                "problemas_ser_atendido_si",
                "problemas_conseguir_cita_si",
                "problemas_llegar_consulta_si",
                "problemas_pagar_por_costo_si",
                "iniciativas_de_inversion",
                "top_5_dias_hospitalizacion_total_parto",
                "top_5_dias_hospitalizacion_total_diabetes_mellitus",
                "top_5_cantidad_hospitalizaciones_influenza_[gripe]_y_neumonía",
                "top_5_cantidad_hospitalizaciones_otras_formas_de_enfermedad_del_corazón",
                "top_5_cantidad_hospitalizaciones_trastornos_de_la_vesícula_biliar_de_las_vías_biliares_y_del_páncreas",
            ]
        ],
        scaled_df,
        pca_df,
    ],
    axis=1,
)

anio_servicio_cluster_df

In [ ]:
fig = px.scatter(
    anio_servicio_cluster_df,
    x="total_gasto",
    y="suicidio_ideacion",
    color="cluster",
    hover_data=["año", "nombre_capitulo", "total_fonasa"],
    title="Total Gasto vs. Ideación Suicida (ingresos 3ra edad)",
)
fig.update_layout(
    height=800,
    width=1400,
)
fig.update_traces(marker=dict(size=8))
fig.show()
fig.write_html("../plots/presentacion/total_gasto_vs_suicidio_ideacion.html")

In [ ]:
anio_servicio_cluster_df.query("año == 2018")["suicidio_intento"].value_counts()

In [ ]:
fig = px.scatter(
    anio_servicio_cluster_df,
    x="total_gasto",
    y="suicidio_intento",
    color="cluster",
    hover_data=["año", "nombre_capitulo", "total_fonasa"],
    title="Total Gasto vs. Intento de Suicidio (ingresos 3ra edad)",
)
fig.update_layout(
    height=800,
    width=1400,
)
fig.update_traces(marker=dict(size=8))
fig.show()
fig.write_html("../plots/presentacion/total_gasto_vs_suicidio_intento.html")

In [ ]:
fig = px.bar(
    anio_servicio_cluster_df.sort_values(by="total_gasto", ascending=True),
    x="total_gasto",
    y="nombre_capitulo",
    color="cluster",
    hover_data=["año", "total_fonasa"],
    title="Total de gasto por Servicio de Salud (ordenado por gasto)",
)
fig.update_layout(
    height=800,
    width=1400,
)
fig.show()
fig.write_html("../plots/presentacion/total_gasto_vs_servicio.html")

In [ ]:
fig = px.bar(
    anio_servicio_cluster_df.sort_values(by="total_fonasa", ascending=True),
    x="total_fonasa",
    y="nombre_capitulo",
    color="cluster",
    hover_data=["año"],
    title="Total de personas en Fonasa por Servicio de Salud (ordenado por personas Fonasa)",
)
fig.update_layout(
    height=800,
    width=1400,
)
fig.show()
fig.write_html("../plots/presentacion/total_fonasa_vs_servicio.html")

In [ ]:
fig = px.bar(
    anio_servicio_cluster_df.sort_values(by="total_fonasa", ascending=True),
    x="urgencias_lesiones_autoinflingidas",
    y="nombre_capitulo",
    color="cluster",
    hover_data=["año", "total_gasto"],
    title="Ingresos 3ra Edad por Urgencias Lesiones Autoinflingidas por Servicio (ordenado por personas Fonasa)",
)
fig.update_layout(
    height=800,
    width=1400,
)
fig.show()
fig.write_html("../plots/presentacion/urgencias_autolesiones_vs_servicio.html")

In [ ]:
fig = px.bar(
    anio_servicio_cluster_df.query("año == 2023").sort_values(
        by="total_fonasa", ascending=True
    ),
    x="urgencias_lesiones_autoinflingidas_scaled",
    y="nombre_capitulo",
    color="cluster",
    hover_data=["año", "total_gasto"],
    title="Año 2023 - Total Gasto vs. Ingresos 3ra Edad por Urgencias Lesiones Autoinflingidas (ordenado por personas Fonasa)",
)
fig.update_layout(
    height=800,
    width=1400,
)
fig.show()
fig.write_html("../plots/presentacion/urgencias_autolesiones_vs_servicio_2023.html")

In [ ]:
anio_servicio_cluster_df.columns

In [ ]:
fig = px.scatter(
    anio_servicio_cluster_df.query("año == 2022"),
    x="urgencias_lesiones_autoinflingidas",
    y="total_gasto",
    color="cluster",
    hover_data=["año", "nombre_capitulo", "total_fonasa"],
    title="Año 2022 - Total Gasto vs. Ingresos 3ra Edad por Urgencias Lesiones Autoinflingidas",
)
fig.update_layout(
    height=800,
    width=1400,
)
fig.update_traces(marker=dict(size=16))
fig.show()
fig.write_html("../plots/presentacion/total_gasto_vs_urgencias_autolesiones_2022.html")

In [ ]:
fig = px.scatter(
    anio_servicio_cluster_df.query("año == 2023"),
    x="urgencias_lesiones_autoinflingidas",
    y="total_gasto",
    color="cluster",
    hover_data=["año", "nombre_capitulo", "total_fonasa"],
    title="Año 2023 - Total Gasto vs. Ingresos 3ra Edad por Urgencias Lesiones Autoinflingidas",
)
fig.update_layout(
    height=800,
    width=1400,
)
fig.update_traces(marker=dict(size=16))
fig.show()
fig.write_html("../plots/presentacion/total_gasto_vs_urgencias_autolesiones_2023.html")

In [ ]:
fig = px.scatter(
    anio_servicio_cluster_df,
    x="suicidio_intento_scaled",
    y="total_gasto",
    color="cluster",
    hover_data=["año", "nombre_capitulo", "total_fonasa"],
    title="Total Gasto vs. Ingresos 3ra Edad por Intento de Suicidio",
)
fig.update_layout(
    height=800,
    width=1400,
)
fig.update_traces(marker=dict(size=16))
fig.show()
fig.write_html("../plots/presentacion/total_gasto_vs_suicidio_intento.html")

In [ ]:
fig = px.scatter(
    anio_servicio_cluster_df,
    x="problemas_conseguir_cita_si",
    y="total_gasto",
    color="cluster",
    hover_data=["año", "nombre_capitulo", "total_fonasa"],
    title="Problemas para conseguir cita vs. Total Gasto",
)
fig.update_layout(
    height=800,
    width=1400,
)
fig.update_traces(marker=dict(size=10))
fig.show()
fig.write_html("../plots/presentacion/problemas_conseguir_cita_vs_total_gasto.html")

In [ ]:
fig = px.scatter(
    anio_servicio_cluster_df,
    x="problemas_conseguir_cita_si",
    y="gastos_en_personal",
    color="cluster",
    hover_data=["año", "nombre_capitulo", "total_fonasa"],
    title="Problemas para conseguir cita vs. Gastos en personal",
)
fig.update_layout(
    height=800,
    width=1400,
)
fig.update_traces(marker=dict(size=10))
fig.show()
fig.write_html(
    "../plots/presentacion/problemas_conseguir_cita_vs_gastos_en_personal.html"
)

In [ ]:
fig = px.bar(
    anio_servicio_cluster_df.sort_values(by="total_fonasa", ascending=True),
    x="problemas_conseguir_cita_si",
    y="nombre_capitulo",
    color="cluster",
    hover_data=["año", "total_fonasa"],
    title="Problemas para conseguir citas por Servicio de Salud",
)
fig.update_layout(
    height=800,
    width=1400,
)
fig.show()

In [ ]:
fig = px.bar(
    anio_servicio_cluster_df.sort_values(
        by="problemas_pagar_por_costo_si", ascending=True
    ),
    x="problemas_pagar_por_costo_si",
    y="nombre_capitulo",
    color="cluster",
    hover_data=["año", "total_fonasa"],
    title="Problemas para pagar por costo por Servicio de Salud",
)
fig.update_layout(
    height=800,
    width=1400,
)
fig.show()

In [ ]:
fig = px.histogram(
    anio_servicio_cluster_df,
    x="problemas_pagar_por_costo_si",
    color="cluster",
    hover_data=["año", "total_fonasa"],
    title="Frecuencia de Problemas para pagar por costo",
)
fig.update_layout(
    height=800,
    width=1400,
)
fig.show()
fig.write_html("../plots/presentacion/frecuencia_problemas_pago.html")

In [ ]:
anio_servicio_cluster_df["problemas_pagar_por_costo_si"].value_counts(
    ascending=True
).sort_index()

In [ ]:
fig = px.histogram(
    anio_servicio_cluster_df,
    x="top_5_cantidad_hospitalizaciones_trastornos_de_la_vesícula_biliar_de_las_vías_biliares_y_del_páncreas",
    color="cluster",
    hover_data=["año", "total_fonasa"],
    title="Frecuencia de hospitalizaciones por Trastornos de la Vesícula Biliar, de las Vías Biliares y del Páncreas",
)
fig.update_layout(
    height=800,
    width=1400,
)
fig.show()
fig.write_html("../plots/presentacion/frecuencia_hospitalizaciones_vesicula.html")

In [ ]:
fig = px.scatter(
    anio_servicio_cluster_df,
    x="iniciativas_de_inversion",
    y="suicidio_ideacion",
    color="cluster",
    hover_data=["año", "nombre_capitulo", "total_fonasa"],
    title="Gasto en Iniciativas de Inversión vs. Ideación Suicida (ingresos 3ra edad)",
)
fig.update_layout(
    height=800,
    width=1400,
)
fig.update_traces(marker=dict(size=8))
fig.show()

In [ ]:
# EXCLUDED_VARIABLES = ["año", "capitulo", "nombre_capitulo"]


# def generate_scatter_matrix_year(df, year):
#     analysis_variables = list(filter(lambda x: x not in EXCLUDED_VARIABLES, df.columns))

#     plot = sns.pairplot(
#         df[df["año"] == year][analysis_variables],
#         kind="reg",
#         diag_kind="kde",
#         plot_kws={"line_kws": {"color": "red"}},
#         diag_kws={"color": "red"},
#     )

#     for ax in plot.axes.flatten():
#         ax.set_xlabel(ax.get_xlabel(), rotation=45)
#         ax.set_ylabel(ax.get_ylabel(), rotation=45)
#         ax.yaxis.get_label().set_horizontalalignment("right")

#     plot.figure.suptitle(f"AÑO {year}", y=1.01, fontsize=16)
#     plot.savefig(f"../plots/clustering/corr_matrix_{year}.png", dpi=65)